# Word2Vec


In [107]:
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
  
warnings.filterwarnings(action = 'ignore')
import string
import nltk
import gensim
from gensim.models import Word2Vec
import re
import pandas as pd 
from nltk.corpus import stopwords

--------------------

In [82]:
#  input file

df = pd.read_csv('../data/processed/author-parse-articles.csv')

# Getting Domains From News Websites 
m = df['art_article_url'].str.extract('(?<=http://)(.*?)(?=/)|(?<=https://)(.*?)(?=/)')
m = m[0].fillna(m[1]).fillna(df['art_article_url'])
df['art_extracted_domain'] = m
df.head()

,Unnamed: 0,art_Unnamed: 0,art_uid,art_article_author,art_article_date,art_article_text,art_article_url,ref_Unnamed: 0,ref_Unnamed: 0.1,ref_uid,...,ref_time,ref_source,ref_time-capture,ref_sc,ref_domain,ref_full-text,ref_sc2,ref_geo,ref_time_parsed,art_extracted_domain
0,0,5,02b5cecd-fac3-4b89-80a2-5bc7d3356d51,['Jake Epstein'],2022-04-04 00:00:00,Lithuania's top diplomat said buying Russian o...,https://www.businessinsider.com/lithuania-dipl...,16,16,02b5cecd-fac3-4b89-80a2-5bc7d3356d51,...,4 hours ago,https://twitter.com/idreesali114/status/151098...,2022-04-04 13:47:10.051481,False,"data:image/svg+xml;base64,PHN2ZyB2ZXJzaW9uPSIx...",SourceOn live map\nTell friends\n4 hours ago -...,False,54°41′N 25°16′E,2022-04-04 09:50:58,www.businessinsider.com
1,1,1,8ea2c08f-d9ff-4aa8-b710-9b6f26c7bb39,"['Martin Belam', 'Léonie Chao-Fong', 'Gloria O...",2022-04-04 00:00:00,"This live blog is now closed, you can follow a...",https://www.theguardian.com/world/live/2022/ap...,17,17,8ea2c08f-d9ff-4aa8-b710-9b6f26c7bb39,...,4 hours ago,https://twitter.com/prm_ua/status/151097769528...,2022-04-04 13:47:14.411519,False,"data:image/svg+xml;base64,PHN2ZyB2ZXJzaW9uPSIx...",SourceOn live map\nTell friends\n4 hours ago -...,False,50°38′N 29°55′E,2022-04-04 09:50:58,www.theguardian.com
2,2,2,8ea2c08f-d9ff-4aa8-b710-9b6f26c7bb39,['Writer Smith'],2022-04-04 17:59:18+00:00,The Prosecutor General’s Office was able to re...,https://news84media.com/russia-ukraine/the-wor...,17,17,8ea2c08f-d9ff-4aa8-b710-9b6f26c7bb39,...,4 hours ago,https://twitter.com/prm_ua/status/151097769528...,2022-04-04 13:47:14.411519,False,"data:image/svg+xml;base64,PHN2ZyB2ZXJzaW9uPSIx...",SourceOn live map\nTell friends\n4 hours ago -...,False,50°38′N 29°55′E,2022-04-04 09:50:58,news84media.com
3,3,3,8ea2c08f-d9ff-4aa8-b710-9b6f26c7bb39,['Scott Lucas'],2022-04-04 06:24:08+01:00,Larisa Savenko outside her damaged house in Bu...,https://eaworldview.com/2022/04/ukraine-war-bu...,17,17,8ea2c08f-d9ff-4aa8-b710-9b6f26c7bb39,...,4 hours ago,https://twitter.com/prm_ua/status/151097769528...,2022-04-04 13:47:14.411519,False,"data:image/svg+xml;base64,PHN2ZyB2ZXJzaW9uPSIx...",SourceOn live map\nTell friends\n4 hours ago -...,False,50°38′N 29°55′E,2022-04-04 09:50:58,eaworldview.com
4,4,4,8ea2c08f-d9ff-4aa8-b710-9b6f26c7bb39,"['Iryna Khalip', 'Natallia Radzina', 'Andrei S...",2022-04-04 00:00:00,AFU Repels Enemy Attacks On All Fronts 8 4.04....,https://charter97.org/en/news/2022/4/4/462037/,17,17,8ea2c08f-d9ff-4aa8-b710-9b6f26c7bb39,...,4 hours ago,https://twitter.com/prm_ua/status/151097769528...,2022-04-04 13:47:14.411519,False,"data:image/svg+xml;base64,PHN2ZyB2ZXJzaW9uPSIx...",SourceOn live map\nTell friends\n4 hours ago -...,False,50°38′N 29°55′E,2022-04-04 09:50:58,charter97.org


In [68]:
df['art_extracted_domain']

0       www.businessinsider.com
1           www.theguardian.com
2               news84media.com
3               eaworldview.com
4                 charter97.org
                 ...           
6342         www.compuserve.com
6343         www.denverpost.com
6344       hindustannewshub.com
6345      albaniandailynews.com
6346                theprint.in
Name: art_extracted_domain, Length: 6347, dtype: object

In [69]:
article = df.loc[df['art_extracted_domain']=='www.compuserve.com']

In [70]:
article

,Unnamed: 0,art_Unnamed: 0,art_uid,art_article_author,art_article_date,art_article_text,art_article_url,ref_Unnamed: 0,ref_Unnamed: 0.1,ref_uid,...,ref_time,ref_source,ref_time-capture,ref_sc,ref_domain,ref_full-text,ref_sc2,ref_geo,ref_time_parsed,art_extracted_domain
6342,6342,8,436ee9aa-945f-4ed3-9566-a0b8f0f018d8,['The Associated Press'],2022-04-06,Live Updates | Russia-Ukraine War\n\nWASHINGTO...,https://www.compuserve.com/pf/story/0001/20220...,14,14,436ee9aa-945f-4ed3-9566-a0b8f0f018d8,...,4 hours ago,https://twitter.com/shashj/status/151098248417...,2022-04-04 13:46:59.387389,False,"data:image/svg+xml;base64,PHN2ZyB2ZXJzaW9uPSIx...",SourceOn live map\nTell friends\n4 hours ago -...,False,59°54′N 10°44′E,2022-04-04 09:50:58,www.compuserve.com


In [71]:
article['art_article_text']

6342    Live Updates | Russia-Ukraine War\n\nWASHINGTO...
Name: art_article_text, dtype: object

In [83]:
type(article['art_article_text'])

pandas.core.series.Series

# preprocess the data

In [84]:
article = article['art_article_text'].to_frame()

In [92]:
type(article['art_article_text'])
art_text = article['art_article_text'].to_frame()

In [93]:
article['art_article_text']= article['art_article_text'].apply(lambda x: x.lower())
article['art_article_text'] =article['art_article_text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '' , x))
stop_words = set(stopwords.words('english'))
stop_words.add('subject')
stop_words.add('http')

def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])

article['art_article_text'] = article['art_article_text'].apply(lambda x: remove_stopwords(x))




In [97]:
article['art_article_text'] = article['art_article_text'].apply(lambda text: re.sub(' +', ' ', text))

In [98]:
article['art_article_text']

6342    live updates russiaukraine war washington — us...
Name: art_article_text, dtype: object

In [99]:
pre_article = article['art_article_text'].replace("\n", " ")

In [110]:
pre_article

all_sentences = nltk.sent_tokenize(str(pre_article))

all_words = [nltk.word_tokenize(sent) for sent in all_sentences]

for i in range(len(all_words)):
    all_words[i] = [w for w in all_words[i] if w not in stopwords.words('english')]

In [119]:
all_words

[['6342',
  'live',
  'updates',
  'russiaukraine',
  'war',
  'washington',
  '—',
  'us',
  '...',
  'Name',
  ':',
  'art_article_text',
  ',',
  'dtype',
  ':',
  'object']]

------------------

In [105]:
data = []
  
# iterate through each sentence in the file
for i in sent_tokenize(str(pre_article)):
    temp = []
      
    # tokenize the sentence into words
    for j in word_tokenize(i):
        temp.append(j.lower())
  
    data.append(temp)
# Create CBOW model
model1 = gensim.models.Word2Vec(data, min_count = 1, 
                              vector_size = 100, window = 5)
  
# Print results
print("Cosine similarity between 'washington' " + 
               "and 'russiaukraine' - CBOW : ",
    model1.wv.most_similar('washington', 'russiaukraine'))
      
print("Cosine similarity between 'washington' " +
                 "and 'war' - CBOW : ",
      model1.wv.most_similar('washington', 'war'))
  
# Create Skip Gram model
model2 = gensim.models.Word2Vec(data, min_count = 1, vector_size = 100,
                                             window = 5, sg = 1)
  
# Print results
print("Cosine similarity between 'russiaukraine' " +
          "and 'war' - Skip Gram : ",
    model2.wv.most_similar('russiaukraine', 'war'))
      
print("Cosine similarity between 'russiaukraine' " +
            "and 'live' - Skip Gram : ",
      model2.wv.most_similar('russiaukraine', 'live'))

Cosine similarity between 'washington' and 'russiaukraine' - CBOW :  [(':', 0.09315099567174911), ('...', 0.05849593132734299), ('live', 0.038677141070365906), ('6342', 0.009206998161971569), ('—', 0.004760038107633591), ('updates', -0.010427212342619896), ('war', -0.06439965963363647), ('object', -0.07099064439535141), ('art_article_text', -0.11415775120258331), ('us', -0.1395045816898346)]
Cosine similarity between 'washington' and 'war' - CBOW :  [(':', 0.109239362180233), ('art_article_text', 0.06103111058473587), ('dtype', 0.04926054924726486), ('object', 0.02743697352707386), ('—', 0.01274148654192686), ('6342', 0.0006383356521837413), ('...', -0.034651242196559906), ('live', -0.0748080238699913), ('name', -0.08916793763637543), ('updates', -0.12377215176820755)]
Cosine similarity between 'russiaukraine' and 'war' - Skip Gram :  [('dtype', 0.2058384269475937), ('art_article_text', 0.18956248462200165), ('name', 0.11010506749153137), ('object', 0.10678696632385254), (',', 0.021225

In [135]:
word2vec = Word2Vec(all_words, min_count=2)

In [136]:
vocabulary = word2vec.wv.index_to_key

In [134]:
# can be used to find the similar words
sim_words = word2vec.wv.most_similar('war')